In [2]:
md_df = pd.read_csv('../raw_data/archive/HAM10000_metadata.csv')

In [3]:
md_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [4]:
image_list = []

for x in glob(os.path.join("../raw_data/archive/HAM10000_all_images", "*.jpg")):
    image_list.append(x)

image_list

['../raw_data/archive/HAM10000_all_images/ISIC_0030858.jpg',
 '../raw_data/archive/HAM10000_all_images/ISIC_0030680.jpg',
 '../raw_data/archive/HAM10000_all_images/ISIC_0033389.jpg',
 '../raw_data/archive/HAM10000_all_images/ISIC_0032097.jpg',
 '../raw_data/archive/HAM10000_all_images/ISIC_0032929.jpg',
 '../raw_data/archive/HAM10000_all_images/ISIC_0026784.jpg',
 '../raw_data/archive/HAM10000_all_images/ISIC_0028971.jpg',
 '../raw_data/archive/HAM10000_all_images/ISIC_0026948.jpg',
 '../raw_data/archive/HAM10000_all_images/ISIC_0026790.jpg',
 '../raw_data/archive/HAM10000_all_images/ISIC_0028965.jpg',
 '../raw_data/archive/HAM10000_all_images/ISIC_0025299.jpg',
 '../raw_data/archive/HAM10000_all_images/ISIC_0032083.jpg',
 '../raw_data/archive/HAM10000_all_images/ISIC_0024839.jpg',
 '../raw_data/archive/HAM10000_all_images/ISIC_0030694.jpg',
 '../raw_data/archive/HAM10000_all_images/ISIC_0024811.jpg',
 '../raw_data/archive/HAM10000_all_images/ISIC_0030864.jpg',
 '../raw_data/archive/HA

In [5]:
image_df = pd.DataFrame(image_list)

In [6]:
image_df.columns = ['filepath']
image_df.head()

,filepath
0,../raw_data/archive/HAM10000_all_images/ISIC_0...
1,../raw_data/archive/HAM10000_all_images/ISIC_0...
2,../raw_data/archive/HAM10000_all_images/ISIC_0...
3,../raw_data/archive/HAM10000_all_images/ISIC_0...
4,../raw_data/archive/HAM10000_all_images/ISIC_0...


In [7]:
image_df['image_id'] = image_df['filepath'].apply(lambda x: os.path.basename(x).strip('.jpg'))
image_df['image_id'] = image_df['image_id'].astype('string')
image_df.head()

,filepath,image_id
0,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0030858
1,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0030680
2,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0033389
3,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0032097
4,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0032929


In [8]:
image_df.astype(str)

,filepath,image_id
0,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0030858
1,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0030680
2,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0033389
3,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0032097
4,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0032929
...,...,...
10010,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0024834
10011,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0033390
10012,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0030699
10013,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0030841


In [9]:
md_df['image_id'] = md_df['image_id'].astype('string')
md_df.dtypes

lesion_id               object
image_id        string[python]
dx                      object
dx_type                 object
age                    float64
sex                     object
localization            object
dtype: object

In [10]:
image_df.dtypes

filepath            object
image_id    string[python]
dtype: object

In [11]:
md_df.dtypes

lesion_id               object
image_id        string[python]
dx                      object
dx_type                 object
age                    float64
sex                     object
localization            object
dtype: object

In [12]:
image_df.dtypes

filepath            object
image_id    string[python]
dtype: object

In [13]:
#full_df = image_df.join(md_df, on='image_id')
full_df = image_df.merge(md_df, on='image_id', how='left')
full_df.head()

,filepath,image_id,lesion_id,dx,dx_type,age,sex,localization
0,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0030858,HAM_0001813,nv,histo,60.0,male,trunk
1,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0030680,HAM_0002381,nv,follow_up,80.0,male,trunk
2,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0033389,HAM_0000268,nv,histo,10.0,female,trunk
3,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0032097,HAM_0003006,nv,follow_up,55.0,male,back
4,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0032929,HAM_0003321,bkl,histo,75.0,female,face


In [14]:
full_df.isnull().sum()

filepath         0
image_id         0
lesion_id        0
dx               0
dx_type          0
age             57
sex              0
localization     0
dtype: int64

In [15]:
# filling the age data with mean values
full_df['age'] = full_df['age'].fillna((full_df['age'].mean()))

In [16]:
#check to see if there is any null values after filling in the na stuff
full_df.isnull().sum()

filepath        0
image_id        0
lesion_id       0
dx              0
dx_type         0
age             0
sex             0
localization    0
dtype: int64

In [17]:
full_df.head()

,filepath,image_id,lesion_id,dx,dx_type,age,sex,localization
0,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0030858,HAM_0001813,nv,histo,60.0,male,trunk
1,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0030680,HAM_0002381,nv,follow_up,80.0,male,trunk
2,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0033389,HAM_0000268,nv,histo,10.0,female,trunk
3,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0032097,HAM_0003006,nv,follow_up,55.0,male,back
4,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0032929,HAM_0003321,bkl,histo,75.0,female,face


In [20]:
# resize the images, and put them in an array
full_df['image_arr'] = full_df['filepath'].apply(lambda x: np.asarray(Image.open(x).resize((96, 96))))

In [21]:
full_df.head()

,filepath,image_id,lesion_id,dx,dx_type,age,sex,localization,image_arr
0,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0030858,HAM_0001813,nv,histo,60.0,male,trunk,"[[[31, 22, 25], [32, 24, 27], [42, 32, 43], [6..."
1,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0030680,HAM_0002381,nv,follow_up,80.0,male,trunk,"[[[225, 153, 161], [225, 157, 164], [226, 158,..."
2,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0033389,HAM_0000268,nv,histo,10.0,female,trunk,"[[[5, 5, 5], [6, 6, 4], [6, 6, 5], [13, 12, 12..."
3,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0032097,HAM_0003006,nv,follow_up,55.0,male,back,"[[[230, 151, 169], [229, 150, 168], [229, 148,..."
4,../raw_data/archive/HAM10000_all_images/ISIC_0...,ISIC_0032929,HAM_0003321,bkl,histo,75.0,female,face,"[[[120, 88, 106], [119, 85, 101], [120, 86, 10..."


In [37]:
full_df.iloc[499]['image_arr'].shape

(96, 96, 3)

# Encoding